In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("oil_production.csv")

In [3]:
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [4]:
df.head()

,DATEPRD,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_CHOKE_SIZE_P,AVG_WHP_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL
0,2009-01-30,257.442241,105.339191,163.294586,35.298817,94.147654,61.049726,4535.43,649388.07,298.19
1,2009-02-11,261.481233,105.364799,164.352071,34.697693,97.129162,65.804801,4379.88,629307.34,143.54
2,2009-02-20,264.386001,105.410873,166.214057,34.779443,98.171944,64.987169,4509.07,638750.17,108.74
3,2009-02-22,266.709086,105.395959,166.273529,34.054792,100.435557,67.330436,4319.02,612912.62,106.60
4,2009-02-23,266.667137,105.410516,166.513059,34.395363,100.154078,66.986427,4417.66,625514.01,117.37


In [5]:
t = df["DATEPRD"]
df.drop("DATEPRD",axis = 1, inplace = True)

In [6]:
X = df[[ 'AVG_WHP_P', 'AVG_CHOKE_SIZE_P', 'AVG_DOWNHOLE_TEMPERATURE']]
Y = df.iloc[:,8]

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split

In [8]:
#x_train,x_test, y_train , y_test = train_test_split(X,Y, test_size = 0.15,shuffle = False)

In [9]:
x_train = X
y_train = Y

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
x_train = scaler_x.fit_transform(x_train)#.as_matrix())
#x_val = scaler_x.transform(x_val.as_matrix())
#x_test = scaler_x.transform(x_test.as_matrix())

scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1,1))
#y_val = scaler_y.transform(y_val.reshape(-1,1))
#y_test = scaler_y.transform(y_test.reshape(-1,1))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [11]:
X = np.array(x_train)
y = np.array(y_train)

In [12]:
X.shape

(2365, 3)

In [13]:
from sklearn.ensemble import RandomForestRegressor

In [14]:
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go

In [15]:
def train(model):
    x_axis = []
    y_axis = []
    for i, (train_idx, val_idx) in enumerate(tscvs.split(X)):
        print(" \nFold", i)
        X_train_cv = X[train_idx]
        y_train_cv = y[train_idx]
        X_val_cv = X[val_idx]
        y_val_cv = y[val_idx]
        model.fit(X_train_cv, y_train_cv)
        print(model.score(X_val_cv, y_val_cv))
        #fig, ax = plt.subplots()
        #plt.plot(model.predict(X_val_cv))
        #plt.plot(y_val_cv, 'r')
        a, b = val_idx,model.predict(X_val_cv)
        x_axis.append(a)
        y_axis.append(scaler_y.inverse_transform(b.reshape(-1,1)))
    return x_axis , y_axis

In [16]:
model = RandomForestRegressor(random_state=42, criterion = 'mse',
 max_depth= 8,
 max_features='sqrt',
 n_estimators= 200)
tscvs = TimeSeriesSplit(n_splits=5)

In [17]:
a,b = train(model)

 
Fold 0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



-7.51514755896174
 
Fold 1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



-2.555499767523821
 
Fold 2


...

-0.009975848462272952
 
Fold 3


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.42403659109385833
 
Fold 4


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



-7.987911809116886


In [18]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual gas rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [19]:
from sklearn.svm import SVR

In [20]:
model = SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001)

In [21]:
a,b = train(model)

 
Fold 0
-0.42180470518146795
 
Fold 1
-4.824635939149343
 
Fold 2
-0.009917067511898026
 
Fold 3
0.007178256196206267
 
Fold 4
-1.5766506491030312


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [22]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual gas rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [23]:
from sklearn.model_selection import GridSearchCV
svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})

In [24]:
svr.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected.

GridSearchCV(cv=5, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1.0, 10.0, 100.0, 1000.0], 'gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [25]:
svr.best_estimator_

SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [26]:
model = SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001)

In [27]:
a,b = train(model)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



 
Fold 0
-0.42180470518146795
 
Fold 1
-4.824635939149343
 
Fold 2
-0.009917067511898026
 
Fold 3
0.007178256196206267
 
Fold 4
-1.5766506491030312


In [28]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual gas rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [29]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=10, 
                             min_child_weight=1, n_estimators=2000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [30]:
a,b = train(model_xgb)

 
Fold 0
-5.077638256826026
 
Fold 1
-2.5793713233964057
 
Fold 2
-0.10116517631956112
 
Fold 3
0.26454011288321155
 
Fold 4
-2.1217025551743993


In [31]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual oil rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [32]:
import lightgbm as lgb
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=64,
                              learning_rate=0.005, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [33]:
a,b = train(model_lgb)

 
Fold 0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



-4.529966422965212
 
Fold 1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



-1.3200838082821478
 
Fold 2


...

-0.03663075871663768
 
Fold 3


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.35432423986338046
 
Fold 4


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



-1.104802236888625


In [34]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual gas rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [35]:
from sklearn.ensemble import ExtraTreesRegressor
model_ett = ExtraTreesRegressor(n_estimators=200, n_jobs=4, min_samples_split=25,
                            min_samples_leaf=35, max_features='auto')
model_ett.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=35, min_samples_split=25,
          min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=4,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [36]:
a,b = train(model_ett)

 
Fold 0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



-2.5414168215207757
 
Fold 1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



-0.8076841368962353
 
Fold 2


...

-0.6941509193645032
 
Fold 3


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.383182125201339
 
Fold 4


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



-1.7270038981601226


In [37]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual gas rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [38]:
from sklearn.svm import SVR
model = SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001)
a,b = train(model)

 
Fold 0
-0.42180470518146795
 
Fold 1
-4.824635939149343
 
Fold 2
-0.009917067511898026
 
Fold 3
0.007178256196206267
 
Fold 4
-1.5766506491030312


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [39]:
trace0 = go.Scatter(x = a[0], y = b[0].reshape(-1,), name = "Prediction 1")
trace1 = go.Scatter(x = a[1], y = b[1].reshape(-1,), name = "Prediction 2")
trace2 = go.Scatter(x = a[2], y = b[2].reshape(-1,), name = "Prediction 3")
trace3 = go.Scatter(x = a[3], y = b[3].reshape(-1,), name = "Prediction 4")
trace4 = go.Scatter(x = a[4], y = b[4].reshape(-1,), name = "Prediction 5")
trace5 = go.Scatter(x = np.arange(len(y_train)), y = scaler_y.inverse_transform(y_train).reshape(-1,),
                    name = 'Actual gas rate')

layout = go.Layout(
    xaxis=dict(
        #title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        #title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
data =  [trace5,trace0,trace1,trace2,trace3,trace4]
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [40]:
from sklearn.model_selection import RandomizeSearch
clf = xgb.XGBRegressor()
param_grid = {
'silent': [False],
'max_depth': [6, 10, 15, 20],
'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
'gamma': [0, 0.25, 0.5, 1.0],
'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
'n_estimators': [100]}
fit_params = {'eval_metric': 'mse',
'early_stopping_rounds': 10,
'eval_set': [(x_val, y_val)]}
rs_clf = RandomizedSearchCV(clf, param_grid, n_iter=20,
n_jobs=1, verbose=2, cv=5,
fit_params=fit_params,
scoring='neg_log_loss', refit=False, random_state=42)
rs_clf.fit(x_train, y_train)

ImportError: cannot import name 'RandomizeSearch'